# Import

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sn
import tensorflow as tf
import tensorflow_addons as tfa

from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from leven import levenshtein

import glob
import sys
import os
import math
import gc
import sys
import sklearn
import time
import json

# TQDM Progress Bar With Pandas Apply Function
tqdm.pandas()

print(f'Tensorflow Version {tf.__version__}')
print(f'Python Version: {sys.version}')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

Tensorflow Version 2.12.0
Python Version: 3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:40:32) [GCC 12.3.0]


# Config

In [2]:
import random
SEED = 42
def seed_everything(seed=SEED):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()

In [3]:
# If Notebook Is Run By Committing or In Interactive Mode For Development
IS_INTERACTIVE = os.environ['KAGGLE_KERNEL_RUN_TYPE'] == 'Interactive'
# Verbose Setting during training
VERBOSE = 1 if IS_INTERACTIVE else 2
# Global Random Seed
SEED = 42
# Number of Frames to resize recording to
FRAME_LEN = 128
# Number of colonne
N_COLS = 164
# Length of Phrase + EOS Token
MAX_PHRASE_LENGTH = 64
# Global debug flag, takes subset of train
DEBUG = False
# Whether to use 10% of data for validation
USE_VAL = False
# Batch Size
BATCH_SIZE = 1024
# Whether to Load Pretrained Weights
LOAD_WEIGHTS = True
#PAD 
pad_token = '^'
pad_token_idx = 59
#Input
INPUT_SHAPE = [128, 276]

# Load

In [4]:
with open ("/kaggle/input/asl-fingerspelling/character_to_prediction_index.json", "r") as f:
    char_to_num = json.load(f)

pad_token = '^'
pad_token_idx = 59

char_to_num[pad_token] = pad_token_idx

num_to_char = {j:i for i,j in char_to_num.items()}
df = pd.read_csv('/kaggle/input/asl-fingerspelling/train.csv')

LIP = [
    61, 185, 40, 39, 37, 0, 267, 269, 270, 409,
    291, 146, 91, 181, 84, 17, 314, 405, 321, 375,
    78, 191, 80, 81, 82, 13, 312, 311, 310, 415,
    95, 88, 178, 87, 14, 317, 402, 318, 324, 308,
]
LPOSE = [13, 15, 17, 19, 21]
RPOSE = [14, 16, 18, 20, 22]
POSE = LPOSE + RPOSE

X = [f'x_right_hand_{i}' for i in range(21)] + [f'x_left_hand_{i}' for i in range(21)] + [f'x_pose_{i}' for i in POSE] + [f'x_face_{i}' for i in LIP]
Y = [f'y_right_hand_{i}' for i in range(21)] + [f'y_left_hand_{i}' for i in range(21)] + [f'y_pose_{i}' for i in POSE] + [f'y_face_{i}' for i in LIP]
Z = [f'z_right_hand_{i}' for i in range(21)] + [f'z_left_hand_{i}' for i in range(21)] + [f'z_pose_{i}' for i in POSE] + [f'z_face_{i}' for i in LIP]

SEL_COLS = X + Y + Z
FRAME_LEN = 128
MAX_PHRASE_LENGTH = 64

LIP_IDX_X   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "x" in col]
RHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "x" in col]
LHAND_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "x" in col]
RPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "x" in col]
LPOSE_IDX_X = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "x" in col]

LIP_IDX_Y   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "y" in col]
RHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "y" in col]
LHAND_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "y" in col]
RPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "y" in col]
LPOSE_IDX_Y = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "y" in col]

LIP_IDX_Z   = [i for i, col in enumerate(SEL_COLS)  if  "face" in col and "z" in col]
RHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if "right" in col and "z" in col]
LHAND_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "left" in col and "z" in col]
RPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in RPOSE and "z" in col]
LPOSE_IDX_Z = [i for i, col in enumerate(SEL_COLS)  if  "pose" in col and int(col[-2:]) in LPOSE and "z" in col]

RHM = np.load("/kaggle/input/personnal-data-3/mean_std/rh_mean.npy")
LHM = np.load("/kaggle/input/personnal-data-3/mean_std/lh_mean.npy")
RPM = np.load("/kaggle/input/personnal-data-3/mean_std/rp_mean.npy")
LPM = np.load("/kaggle/input/personnal-data-3/mean_std/lp_mean.npy")
LIPM = np.load("/kaggle/input/personnal-data-3/mean_std/lip_mean.npy")

RHS = np.load("/kaggle/input/personnal-data-3/mean_std/rh_std.npy")
LHS = np.load("/kaggle/input/personnal-data-3/mean_std/lh_std.npy")
RPS = np.load("/kaggle/input/personnal-data-3/mean_std/rp_std.npy")
LPS = np.load("/kaggle/input/personnal-data-3/mean_std/lp_std.npy")
LIPS = np.load("/kaggle/input/personnal-data-3/mean_std/lip_std.npy")

# Model

In [5]:
tffiles = [f"/kaggle/input/personnal-data-3/tfds/{file_id}.tfrecord" for file_id in df.file_id.unique()]
val_len = int(0.05 * len(tffiles))
print('val_len: ' + str(val_len))
train_batch_size = 1024
val_batch_size = 1024

val_len: 3


In [6]:
def load_relevant_data_subset(pq_path):
    return pd.read_parquet(pq_path, columns=SEL_COLS)

file_id = df.file_id.iloc[0]
inpdir = "/kaggle/input/asl-fingerspelling/train_landmarks"
pqfile = f"{inpdir}/{file_id}.parquet"
seq_refs = df.loc[df.file_id == file_id]
seqs = load_relevant_data_subset(pqfile)

seq_id = seq_refs.sequence_id.iloc[0]
frames = seqs.iloc[seqs.index == seq_id]
phrase = str(df.loc[df.sequence_id == seq_id].phrase.iloc[0])

In [7]:
@tf.function()
def tf_nan_mean(x, axis=0, keepdims=False):

    return tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), x), axis=axis, keepdims=keepdims) / tf.reduce_sum(tf.where(tf.math.is_nan(x), tf.zeros_like(x), tf.ones_like(x)), axis=axis, keepdims=keepdims)

 
@tf.function()
def tf_nan_std(x, center=None, axis=0, keepdims=False):

    if center is None:

        center = tf_nan_mean(x, axis=axis,  keepdims=True)

    d = x - center

    return tf.math.sqrt(tf_nan_mean(d * d, axis=axis, keepdims=keepdims))

@tf.function()
def resize_pad(x):
    if tf.shape(x)[0] < FRAME_LEN:
        x = tf.pad(x, ([[0, FRAME_LEN-tf.shape(x)[0]], [0, 0], [0, 0]]), constant_values=float(-1000.0))
    else:
        x = tf.image.resize(x, (FRAME_LEN, tf.shape(x)[1]))
    return x

@tf.function(jit_compile=True)
def pre_process0(x):
    lip_x = tf.gather(x, LIP_IDX_X, axis=1)
    lip_y = tf.gather(x, LIP_IDX_Y, axis=1)
    lip_z = tf.gather(x, LIP_IDX_Z, axis=1)

    rhand_x = tf.gather(x, RHAND_IDX_X, axis=1)
    rhand_y = tf.gather(x, RHAND_IDX_Y, axis=1)
    rhand_z = tf.gather(x, RHAND_IDX_Z, axis=1)
    
    lhand_x = tf.gather(x, LHAND_IDX_X, axis=1)
    lhand_y = tf.gather(x, LHAND_IDX_Y, axis=1)
    lhand_z = tf.gather(x, LHAND_IDX_Z, axis=1)

    rpose_x = tf.gather(x, RPOSE_IDX_X, axis=1)
    rpose_y = tf.gather(x, RPOSE_IDX_Y, axis=1)
    rpose_z = tf.gather(x, RPOSE_IDX_Z, axis=1)
    
    lpose_x = tf.gather(x, LPOSE_IDX_X, axis=1)
    lpose_y = tf.gather(x, LPOSE_IDX_Y, axis=1)
    lpose_z = tf.gather(x, LPOSE_IDX_Z, axis=1)
    
    lip   = tf.concat([lip_x[..., tf.newaxis], lip_y[..., tf.newaxis], lip_z[..., tf.newaxis]], axis=-1)
    rhand = tf.concat([rhand_x[..., tf.newaxis], rhand_y[..., tf.newaxis], rhand_z[..., tf.newaxis]], axis=-1)
    lhand = tf.concat([lhand_x[..., tf.newaxis], lhand_y[..., tf.newaxis], lhand_z[..., tf.newaxis]], axis=-1)
    rpose = tf.concat([rpose_x[..., tf.newaxis], rpose_y[..., tf.newaxis], rpose_z[..., tf.newaxis]], axis=-1)
    lpose = tf.concat([lpose_x[..., tf.newaxis], lpose_y[..., tf.newaxis], lpose_z[..., tf.newaxis]], axis=-1)
#     hand = tf.concat([rhand, lhand], axis=1)
#     hand = tf.where(tf.math.is_nan(hand), 0.0, hand)
#     mask = tf.math.not_equal(tf.reduce_sum(hand, axis=[1, 2]), 0.0)

#     lip = lip[mask]
#     rhand = rhand[mask]
#     lhand = lhand[mask]
#     rpose = rpose[mask]
#     lpose = lpose[mask]
    lip = tf.where(tf.math.is_nan(lip), 0.0, lip)
    rhand = tf.where(tf.math.is_nan(rhand), 0.0, rhand)
    lhand = tf.where(tf.math.is_nan(lhand), 0.0, lhand)
    rpose = tf.where(tf.math.is_nan(rpose), 0.0, rpose)
    lpose = tf.where(tf.math.is_nan(lpose), 0.0, lpose)

    return lip, rhand, lhand, rpose, lpose

@tf.function()
# def pre_process1(lip, rhand, lhand, rpose, lpose):
#     lip   = (resize_pad(lip) - LIPM) / LIPS
#     rhand = (resize_pad(rhand) - RHM) / RHS
#     lhand = (resize_pad(lhand) - LHM) / LHS
#     rpose = (resize_pad(rpose) - RPM) / RPS
#     lpose = (resize_pad(lpose) - LPM) / LPS

#     x = tf.concat([lip, rhand, lhand, rpose, lpose], axis=1)
#     s = tf.shape(x)
#     x = tf.reshape(x, (s[0], s[1]*s[2]))
#     x = tf.where(tf.math.is_nan(x), 0.0, x)
#     return x

def pre_process1(lip, rhand, lhand, rpose, lpose):
    lip   = (lip - tf_nan_mean(lip, keepdims=True)) / (tf_nan_std(lip, keepdims=True))
    rhand = (rhand - tf_nan_mean(rhand, keepdims=True)) / (tf_nan_std(rhand, keepdims=True))
    lhand = (lhand - tf_nan_mean(lhand, keepdims=True)) / (tf_nan_std(lhand, keepdims=True))
    rpose = (rpose - tf_nan_mean(rpose, keepdims=True)) / (tf_nan_std(rpose, keepdims=True))
    lpose = (lpose - tf_nan_mean(lpose, keepdims=True)) / (tf_nan_std(lpose, keepdims=True))

    x = tf.concat([lip, rhand, lhand, rpose, lpose], axis=1)
    x = resize_pad(x)
#     s = tf.shape(x)
#     x = tf.reshape(x, (s[0], s[1]*s[2]))
    x = tf.unstack(x, axis=-1)
    x = tf.concat(x, axis=-1)
    
    x = tf.where(tf.math.is_nan(x), 0.0, x)
    return x

pre0 = pre_process0(frames)
pre1 = pre_process1(*pre0)
INPUT_SHAPE = list(pre1.shape)
print(INPUT_SHAPE)
#pre1
#INPUT_SHAPE = [128, 276]

[128, 276]


In [8]:
def decode_fn(record_bytes):
    schema = {
        "lip": tf.io.VarLenFeature(tf.float32),
        "rhand": tf.io.VarLenFeature(tf.float32),
        "lhand": tf.io.VarLenFeature(tf.float32),
        "rpose": tf.io.VarLenFeature(tf.float32),
        "lpose": tf.io.VarLenFeature(tf.float32),
        "phrase": tf.io.VarLenFeature(tf.int64)
    }
    x = tf.io.parse_single_example(record_bytes, schema)

    lip = tf.reshape(tf.sparse.to_dense(x["lip"]), (-1, 40, 3))
    rhand = tf.reshape(tf.sparse.to_dense(x["rhand"]), (-1, 21, 3))
    lhand = tf.reshape(tf.sparse.to_dense(x["lhand"]), (-1, 21, 3))
    rpose = tf.reshape(tf.sparse.to_dense(x["rpose"]), (-1, 5, 3))
    lpose = tf.reshape(tf.sparse.to_dense(x["lpose"]), (-1, 5, 3))
    phrase = tf.sparse.to_dense(x["phrase"])

    return lip, rhand, lhand, rpose, lpose, phrase

def pre_process_fn(lip, rhand, lhand, rpose, lpose, phrase):
    phrase = tf.pad(phrase, [[0, MAX_PHRASE_LENGTH-tf.shape(phrase)[0]]], constant_values=pad_token_idx)
    return pre_process1(lip, rhand, lhand, rpose, lpose), phrase
    
tffiles = [f"/kaggle/input/personnal-data-3/tfds/{file_id}.tfrecord" for file_id in df.file_id.unique()]
val_len = 1#int(0.05 * len(tffiles))
print('val_len: ' + str(val_len))
# train_batch_size = 256
# val_batch_size = 256
'''
train_dataset =  tf.data.TFRecordDataset(tffiles).prefetch(tf.data.AUTOTUNE).shuffle(5000).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(train_batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset =  tf.data.TFRecordDataset(tffiles[:val_len]).prefetch(tf.data.AUTOTUNE).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE).batch(train_batch_size).prefetch(tf.data.AUTOTUNE)
'''
train_dataset_pre =  tf.data.TFRecordDataset(tffiles[val_len:]).prefetch(tf.data.AUTOTUNE).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE)
val_dataset_pre =  tf.data.TFRecordDataset(tffiles[:val_len]).prefetch(tf.data.AUTOTUNE).map(decode_fn, num_parallel_calls=tf.data.AUTOTUNE).map(pre_process_fn, num_parallel_calls=tf.data.AUTOTUNE)

'''
batch = next(iter(val_dataset))
batch[0].shape, batch[1].shape
'''

val_len: 1


'\nbatch = next(iter(val_dataset))\nbatch[0].shape, batch[1].shape\n'

In [9]:
# val_items = [x for x in val_dataset_pre]
# val_items_X = [x[0] for x in val_items]
# val_items_y = [tf.cast(x[1], dtype = tf.int32) for x in val_items]
# val_dataset = tf.data.Dataset.from_tensor_slices((val_items_X, val_items_y)).prefetch(tf.data.AUTOTUNE).batch(val_batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# train_items = [x for x in train_dataset_pre]
# train_items_X = [x[0] for x in train_items]
# train_items_y = [tf.cast(x[1], dtype = tf.int32) for x in train_items]
# train_dataset = tf.data.Dataset.from_tensor_slices((train_items_X, train_items_y)).prefetch(tf.data.AUTOTUNE).shuffle(60000).batch(train_batch_size, drop_remainder=True).prefetch(tf.data.AUTOTUNE)

# batch = next(iter(val_dataset))
# batch[0].shape, batch[1].shape

In [10]:
#val_dataset

In [11]:
#batch[0]

In [12]:
# #Copied from previous comp 1st place model: https://www.kaggle.com/code/hoyso48/1st-place-solution-training
# class ECA(tf.keras.layers.Layer):
#     def __init__(self, kernel_size=5,dropout_rate=0.2, **kwargs):
#         super().__init__(**kwargs)
#         self.supports_masking = True
#         self.kernel_size = kernel_size
#         self.conv = tf.keras.layers.Conv1D(1, kernel_size=kernel_size, strides=1, padding="same", use_bias=False)
#         self.dropout = tf.keras.layers.Dropout(dropout_rate)  # Ajout de la couche de dropout
#     def call(self, inputs, mask=None, training=None):  #training pour choisir quand faut activer drop_out (pendant training)
#         nn = tf.keras.layers.GlobalAveragePooling1D()(inputs, mask=mask)
#         nn = tf.expand_dims(nn, -1)
#         nn = self.conv(nn)
#         nn = tf.squeeze(nn, -1)
#         nn = tf.nn.sigmoid(nn)
#         nn = nn[:,None,:]
#         nn = self.dropout(nn, training=training) 

#         return inputs * nn

# class CausalDWConv1D(tf.keras.layers.Layer):
#     def __init__(self, 
#         kernel_size=17,
#         dilation_rate=1,
#         use_bias=False,
#         depthwise_initializer='glorot_uniform',
#         dropout_rate=0.2,  # Ajout du taux de dropout
#         name='', **kwargs):
#         super().__init__(name=name,**kwargs)
#         self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
#         self.dw_conv = tf.keras.layers.DepthwiseConv1D(
#                             kernel_size,
#                             strides=1,
#                             dilation_rate=dilation_rate,
#                             padding='valid',
#                             use_bias=use_bias,
#                             depthwise_initializer=depthwise_initializer,
#                             name=name + '_dwconv')
#         self.dropout = tf.keras.layers.Dropout(dropout_rate)  # Ajout de la couche de dropout
#         self.supports_masking = True
        
#     def call(self, inputs, training=None):
#         x = self.causal_pad(inputs)
#         x = self.dw_conv(x)
#         x = self.dropout(x, training=training)  
#         return x

# def Conv1DBlock(channel_size,
#           kernel_size,
#           dilation_rate=1,
#           drop_rate=0.0,
#           expand_ratio=2,
#           se_ratio=0.25,
#           activation='swish',
#           name=None):
#     '''
#     efficient conv1d block, @hoyso48
#     '''
#     if name is None:
#         name = str(tf.keras.backend.get_uid("mbblock"))
#     # Expansion phase
#     def apply(inputs):
#         channels_in = tf.keras.backend.int_shape(inputs)[-1]
#         channels_expand = channels_in * expand_ratio

#         skip = inputs

#         x = tf.keras.layers.Dense(
#             channels_expand,
#             use_bias=True,
#             activation=activation,
#             name=name + '_expand_conv')(inputs)

#         # Depthwise Convolution
#         x = CausalDWConv1D(kernel_size,
#             dilation_rate=dilation_rate,
#             use_bias=False,
#             name=name + '_dwconv')(x)

#         x = tf.keras.layers.BatchNormalization(momentum=0.95, name=name + '_bn')(x)

#         x  = ECA()(x)

#         x = tf.keras.layers.Dense(
#             channel_size,
#             use_bias=True,
#             name=name + '_project_conv')(x)

#         if drop_rate > 0:
#             x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1), name=name + '_drop')(x)

#         if (channels_in == channel_size):
#             x = tf.keras.layers.add([x, skip], name=name + '_add')
#         return x

#     return apply

# class MultiHeadSelfAttention(tf.keras.layers.Layer):
#     def __init__(self, dim=384, num_heads=4, dropout=0, **kwargs):
#         super().__init__(**kwargs)
#         self.dim = dim
#         self.scale = self.dim ** -0.5
#         self.num_heads = num_heads
#         self.qkv = tf.keras.layers.Dense(3 * dim, use_bias=False)
#         self.drop1 = tf.keras.layers.Dropout(dropout)
#         self.proj = tf.keras.layers.Dense(dim, use_bias=False)
#         self.supports_masking = True

#     def call(self, inputs, mask=None):
#         qkv = self.qkv(inputs)
#         qkv = tf.keras.layers.Permute((2, 1, 3))(tf.keras.layers.Reshape((-1, self.num_heads, self.dim * 3 // self.num_heads))(qkv))
#         q, k, v = tf.split(qkv, [self.dim // self.num_heads] * 3, axis=-1)

#         attn = tf.matmul(q, k, transpose_b=True) * self.scale

#         if mask is not None:
#             mask = mask[:, None, None, :]

#         attn = tf.keras.layers.Softmax(axis=-1)(attn, mask=mask)
#         attn = self.drop1(attn)

#         x = attn @ v
#         x = tf.keras.layers.Reshape((-1, self.dim))(tf.keras.layers.Permute((2, 1, 3))(x))
#         x = self.proj(x)
#         return x


# def TransformerBlock(dim=384, num_heads=8, expand=4, attn_dropout=0.2, drop_rate=0.2, activation='swish'):
#     def apply(inputs):
#         x = inputs
#         x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
#         x = MultiHeadSelfAttention(dim=dim,num_heads=num_heads,dropout=attn_dropout)(x)
#         x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
#         x = tf.keras.layers.Add()([inputs, x])
#         attn_out = x

#         x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
#         x = tf.keras.layers.Dense(dim*expand, use_bias=False, activation=activation)(x)
#         x = tf.keras.layers.Dense(dim, use_bias=False)(x)
#         x = tf.keras.layers.Dropout(drop_rate, noise_shape=(None,1,1))(x)
#         x = tf.keras.layers.Add()([attn_out, x])
#         return x
#     return apply

# def positional_encoding(MAX_PHRASE_LENGTH, num_hid):
#         depth = num_hid/2
#         positions = tf.range(MAX_PHRASE_LENGTH, dtype = tf.float32)[..., tf.newaxis]
#         depths = tf.range(depth, dtype = tf.float32)[np.newaxis, :]/depth
#         angle_rates = tf.math.divide(1, tf.math.pow(tf.cast(10000, tf.float32), depths))
#         angle_rads = tf.linalg.matmul(positions, angle_rates)
#         pos_encoding = tf.concat(
#           [tf.math.sin(angle_rads), tf.math.cos(angle_rads)],
#           axis=-1)
#         return pos_encoding

In [13]:
# import typing

# def shape_list(x, out_type=tf.int32):
#     """Deal with dynamic shape in tensorflow cleanly."""
#     static = x.shape.as_list()
#     dynamic = tf.shape(x, out_type=out_type)
#     return [dynamic[i] if s is None else s for i, s in enumerate(static)]


# def get_shape_invariants(tensor):
#     shapes = shape_list(tensor)
#     return tf.TensorShape([i if isinstance(i, int) else None for i in shapes])


# def get_float_spec(tensor):
#     shape = get_shape_invariants(tensor)
#     return tf.TensorSpec(shape, dtype=tf.float32)

# class GLU(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         axis=-1,
#         name="glu_activation",
#         **kwargs,
#     ):
#         super(GLU, self).__init__(name=name, **kwargs)
#         self.axis = axis

#     def call(
#         self,
#         inputs,
#         **kwargs,
#     ):
#         a, b = tf.split(inputs, 2, axis=self.axis)
#         b = tf.nn.sigmoid(b)
#         return tf.multiply(a, b)

#     def get_config(self):
#         conf = super(GLU, self).get_config()
#         conf.update({"axis": self.axis})
#         return conf

    
# class MultiHeadAttention(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         num_heads,
#         head_size,
#         output_size: int = None,
#         dropout: float = 0.0,
#         use_projection_bias: bool = True,
#         return_attn_coef: bool = False,
#         kernel_initializer: typing.Union[str, typing.Callable] = tf.keras.initializers.glorot_uniform(seed=SEED),
#         kernel_regularizer: typing.Union[str, typing.Callable] = None,
#         kernel_constraint: typing.Union[str, typing.Callable] = None,
#         bias_initializer: typing.Union[str, typing.Callable] = "zeros",
#         bias_regularizer: typing.Union[str, typing.Callable] = None,
#         bias_constraint: typing.Union[str, typing.Callable] = None,
#         **kwargs,
#     ):
#         super(MultiHeadAttention, self).__init__(**kwargs)

#         if output_size is not None and output_size < 1:
#             raise ValueError("output_size must be a positive number")

#         self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
#         self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
#         self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
#         self.bias_initializer = tf.keras.initializers.get(bias_initializer)
#         self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
#         self.bias_constraint = tf.keras.constraints.get(bias_constraint)

#         self.head_size = head_size
#         self.num_heads = num_heads
#         self.output_size = output_size
#         self.use_projection_bias = use_projection_bias
#         self.return_attn_coef = return_attn_coef

#         self.dropout = tf.keras.layers.Dropout(dropout, name="dropout")
#         self._droput_rate = dropout
#         self.supports_masking = True # RAJOUT

#     def build(
#         self,
#         input_shape,
#     ):
#         num_query_features = input_shape[0][-1]
#         num_key_features = input_shape[1][-1]
#         num_value_features = input_shape[2][-1] if len(input_shape) > 2 else num_key_features
#         output_size = self.output_size if self.output_size is not None else num_value_features
#         self.query_kernel = self.add_weight(
#             name="query_kernel",
#             shape=[self.num_heads, num_query_features, self.head_size],
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer,
#             constraint=self.kernel_constraint,
#         )
#         self.key_kernel = self.add_weight(
#             name="key_kernel",
#             shape=[self.num_heads, num_key_features, self.head_size],
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer,
#             constraint=self.kernel_constraint,
#         )
#         self.value_kernel = self.add_weight(
#             name="value_kernel",
#             shape=[self.num_heads, num_value_features, self.head_size],
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer,
#             constraint=self.kernel_constraint,
#         )
#         self.projection_kernel = self.add_weight(
#             name="projection_kernel",
#             shape=[self.num_heads, self.head_size, output_size],
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer,
#             constraint=self.kernel_constraint,
#         )
#         if self.use_projection_bias:
#             self.projection_bias = self.add_weight(
#                 name="projection_bias",
#                 shape=[output_size],
#                 initializer=self.bias_initializer,
#                 regularizer=self.bias_regularizer,
#                 constraint=self.bias_constraint,
#             )
#         else:
#             self.projection_bias = None

#     def call_qkv(
#         self,
#         query,
#         key,
#         value,
#         training=None,
#     ):
#         # verify shapes
#         if key.shape[-2] != value.shape[-2]:
#             raise ValueError(
#                 "the number of elements in 'key' must be equal to " "the same as the number of elements in 'value'"
#             )
#         # Linear transformations
#         query = tf.einsum("...NI,HIO->...NHO", query, self.query_kernel)
#         key = tf.einsum("...MI,HIO->...MHO", key, self.key_kernel)
#         value = tf.einsum("...MI,HIO->...MHO", value, self.value_kernel)

#         return query, key, value

#     def call_attention(
#         self,
#         query,
#         key,
#         value,
#         logits,
#         training=None,
#         mask=None,
#     ):
#         # mask = attention mask with shape [B, Tquery, Tkey] with 1 is for positions we want to attend, 0 for masked
#         if mask is not None:
#             if len(mask.shape) < 2: #was written mask 
#                 raise ValueError("'mask' must have at least 2 dimensions")
#             if query.shape[-3] != mask.shape[-2]:
#                 raise ValueError("mask's second to last dimension must be equal to " "the number of elements in 'query'")
#             if key.shape[-3] != mask.shape[-1]:
#                 raise ValueError("mask's last dimension must be equal to the number of elements in 'key'")
#         # apply mask
#         if mask is not None:
#             mask = tf.cast(mask, tf.float32)

#             # possibly expand on the head dimension so broadcasting works
#             if len(mask.shape) != len(logits.shape):
#                 mask = tf.expand_dims(mask, -3)

#             logits += -10e9 * (1.0 - mask)

#         attn_coef = tf.nn.softmax(logits)

#         # attention dropout
#         attn_coef_dropout = self.dropout(attn_coef, training=training)

#         # attention * value
#         multihead_output = tf.einsum("...HNM,...MHI->...NHI", attn_coef_dropout, value)

#         # Run the outputs through another linear projection layer. Recombining heads
#         # is automatically done.
#         output = tf.einsum("...NHI,HIO->...NO", multihead_output, self.projection_kernel)

#         if self.projection_bias is not None:
#             output += self.projection_bias

#         return output, attn_coef

#     def call(
#         self,
#         inputs,
#         training=None,
#         mask=None,
#         **kwargs,
#     ):
#         query, key, value = inputs

#         query, key, value = self.call_qkv(query, key, value, training=training)

#         # Scale dot-product, doing the division to either query or key
#         # instead of their product saves some computation
#         depth = tf.constant(self.head_size, dtype=tf.float32)
#         query /= tf.sqrt(depth)

#         # Calculate dot product attention
#         logits = tf.einsum("...NHO,...MHO->...HNM", query, key)

#         output, attn_coef = self.call_attention(query, key, value, logits, training=training, mask=mask)

#         if self.return_attn_coef:
#             return output, attn_coef
#         else:
#             return output

#     def compute_output_shape(
#         self,
#         input_shape,
#     ):
#         num_value_features = input_shape[2][-1] if len(input_shape) > 2 else input_shape[1][-1]
#         output_size = self.output_size if self.output_size is not None else num_value_features

#         output_shape = input_shape[0][:-1] + (output_size,)

#         if self.return_attn_coef:
#             num_query_elements = input_shape[0][-2]
#             num_key_elements = input_shape[1][-2]
#             attn_coef_shape = input_shape[0][:-2] + (
#                 self.num_heads,
#                 num_query_elements,
#                 num_key_elements,
#             )

#             return output_shape, attn_coef_shape
#         else:
#             return output_shape

#     def get_config(self):
#         config = super().get_config()

#         config.update(
#             head_size=self.head_size,
#             num_heads=self.num_heads,
#             output_size=self.output_size,
#             dropout=self._droput_rate,
#             use_projection_bias=self.use_projection_bias,
#             return_attn_coef=self.return_attn_coef,
#             kernel_initializer=tf.keras.initializers.serialize(self.kernel_initializer),
#             kernel_regularizer=tf.keras.regularizers.serialize(self.kernel_regularizer),
#             kernel_constraint=tf.keras.constraints.serialize(self.kernel_constraint),
#             bias_initializer=tf.keras.initializers.serialize(self.bias_initializer),
#             bias_regularizer=tf.keras.regularizers.serialize(self.bias_regularizer),
#             bias_constraint=tf.keras.constraints.serialize(self.bias_constraint),
#         )

#         return config


# class RelPositionMultiHeadAttention(MultiHeadAttention):
#     def build(
#         self,
#         input_shape,
#     ):
#         num_pos_features = input_shape[-1][-1]
#         self.pos_kernel = self.add_weight(
#             name="pos_kernel",
#             shape=[self.num_heads, num_pos_features, self.head_size],
#             initializer=self.kernel_initializer,
#             regularizer=self.kernel_regularizer,
#             constraint=self.kernel_constraint,
#         )
#         self.pos_bias_u = self.add_weight(
#             name="pos_bias_u",
#             shape=[self.num_heads, self.head_size],
#             regularizer=self.kernel_regularizer,
#             initializer=self.kernel_initializer,
#             constraint=self.kernel_constraint,
#         )
#         self.pos_bias_v = self.add_weight(
#             name="pos_bias_v",
#             shape=[self.num_heads, self.head_size],
#             regularizer=self.kernel_regularizer,
#             initializer=self.kernel_initializer,
#             constraint=self.kernel_constraint,
#         )
#         super(RelPositionMultiHeadAttention, self).build(input_shape[:-1])

#     @staticmethod
#     def relative_shift(x):
#         x_shape = tf.shape(x)
#         x = tf.pad(x, [[0, 0], [0, 0], [0, 0], [1, 0]])
#         x = tf.reshape(x, [x_shape[0], x_shape[1], x_shape[3] + 1, x_shape[2]])
#         x = tf.reshape(x[:, :, 1:, :], x_shape)
#         return x

#     def call(
#         self,
#         inputs,
#         training=None,
#         mask=None,
#         **kwargs,
#     ):
#         query, key, value, pos = inputs

#         query, key, value = self.call_qkv(query, key, value, training=training)

#         pos = tf.einsum("...MI,HIO->...MHO", pos, self.pos_kernel)

#         query_with_u = query + self.pos_bias_u
#         query_with_v = query + self.pos_bias_v

#         logits_with_u = tf.einsum("...NHO,...MHO->...HNM", query_with_u, key)
#         logits_with_v = tf.einsum("...NHO,...MHO->...HNM", query_with_v, pos)
#         logits_with_v = self.relative_shift(logits_with_v)

#         logits = logits_with_u + logits_with_v[:, :, :, : tf.shape(logits_with_u)[3]]

#         depth = tf.constant(self.head_size, dtype=tf.float32)
#         logits /= tf.sqrt(depth)

#         output, attn_coef = self.call_attention(query, key, value, logits, training=training, mask=mask)

#         if self.return_attn_coef:
#             return output, attn_coef
#         else:
#             return output
        
        
# class PositionalEncoding(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         alpha: int = 1,
#         beta: int = 0,
#         name="positional_encoding",
#         **kwargs,
#     ):
#         super().__init__(trainable=False, name=name, **kwargs)
#         self.alpha = alpha
#         self.beta = beta
#         self.supports_masking = True # RAJOUT

#     def build(
#         self,
#         input_shape,
#     ):
#         dmodel = input_shape[-1]
#         assert dmodel % 2 == 0, f"Input last dim must be even: {dmodel}"

#     @staticmethod
#     def encode(
#         max_len,
#         dmodel,
#     ):
#         pos = tf.expand_dims(tf.range(max_len - 1, -1, -1.0, dtype=tf.float32), axis=1)
#         index = tf.expand_dims(tf.range(0, dmodel, dtype=tf.float32), axis=0)

#         pe = pos * (1 / tf.pow(10000.0, (2 * (index // 2)) / dmodel))

#         # Sin cos will be [max_len, size // 2]
#         # we add 0 between numbers by using padding and reshape
#         sin = tf.pad(tf.expand_dims(tf.sin(pe[:, 0::2]), -1), [[0, 0], [0, 0], [0, 1]], mode="CONSTANT", constant_values=0)
#         sin = tf.reshape(sin, [max_len, dmodel])
#         cos = tf.pad(tf.expand_dims(tf.cos(pe[:, 1::2]), -1), [[0, 0], [0, 0], [1, 0]], mode="CONSTANT", constant_values=0)
#         cos = tf.reshape(cos, [max_len, dmodel])
#         # Then add sin and cos, which results in [time, size]
#         pe = tf.add(sin, cos)
#         return tf.expand_dims(pe, axis=0)  # [1, time, size]

#     def call(
#         self,
#         inputs,
#         **kwargs,
#     ):
#         # inputs shape [B, T, V]
#         _, max_len, dmodel = shape_list(inputs)
#         pe = self.encode(max_len * self.alpha + self.beta, dmodel)
#         return tf.cast(pe, dtype=inputs.dtype)

#     def get_config(self):
#         conf = super().get_config()
#         conf.update({"alpha": self.alpha, "beta": self.beta})
#         return conf


# class PositionalEncodingConcat(PositionalEncoding):
#     def build(
#         self,
#         input_shape,
#     ):
#         dmodel = input_shape[-1]
#         assert dmodel % 2 == 0, f"Input last dim must be even: {dmodel}"

#     @staticmethod
#     def encode(
#         max_len,
#         dmodel,
#     ):
#         pos = tf.range(max_len - 1, -1, -1.0, dtype=tf.float32)

#         index = tf.range(0, dmodel, 2.0, dtype=tf.float32)
#         index = 1 / tf.pow(10000.0, (index / dmodel))

#         sinusoid = tf.einsum("i,j->ij", pos, index)
#         pos = tf.concat([tf.sin(sinusoid), tf.cos(sinusoid)], axis=-1)

#         return tf.expand_dims(pos, axis=0)

#     def call(
#         self,
#         inputs,
#         **kwargs,
#     ):
#         # inputs shape [B, T, V]
#         _, max_len, dmodel = shape_list(inputs)
#         pe = self.encode(max_len * self.alpha + self.beta, dmodel)
#         return tf.cast(pe, dtype=inputs.dtype)
    
    

# L2 = tf.keras.regularizers.l2(1e-6)


# class FFModule(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         input_dim,
#         dropout=0.0,
#         fc_factor=0.5,
#         kernel_regularizer=L2,
#         bias_regularizer=L2,
#         name="ff_module",
#         **kwargs,
#     ):
#         super(FFModule, self).__init__(name=name, **kwargs)
#         self.fc_factor = fc_factor
#         self.ln = tf.keras.layers.LayerNormalization(
#             name=f"{name}_ln",
#             gamma_regularizer=kernel_regularizer,
#             beta_regularizer=bias_regularizer,
#         )
#         self.ffn1 = tf.keras.layers.Dense(
#             4 * input_dim,
#             name=f"{name}_dense_1",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.swish = tf.keras.layers.Activation(tf.nn.swish, name=f"{name}_swish_activation")
#         self.do1 = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout_1")
#         self.ffn2 = tf.keras.layers.Dense(
#             input_dim,
#             name=f"{name}_dense_2",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.do2 = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout_2")
#         self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.4)
#         self.supports_masking = True # RAJOUT

#     def call(
#         self,
#         inputs,
#         training=None,
#         **kwargs,
#     ):
#         outputs = self.ln(inputs, training=training)
#         outputs = self.ffn1(outputs, training=training)
#         outputs = self.swish(outputs)
#         outputs = self.do1(outputs, training=training)
#         outputs = self.ffn2(outputs, training=training)
#         outputs = self.do2(outputs, training=training)
#         outputs = self.res_add([inputs, self.fc_factor * outputs])
#         return outputs

#     def get_config(self):
#         conf = super(FFModule, self).get_config()
#         conf.update({"fc_factor": self.fc_factor})
#         conf.update(self.ln.get_config())
#         conf.update(self.ffn1.get_config())
#         conf.update(self.swish.get_config())
#         conf.update(self.do1.get_config())
#         conf.update(self.ffn2.get_config())
#         conf.update(self.do2.get_config())
#         conf.update(self.res_add.get_config())
#         return conf


# class MHSAModule(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         head_size,
#         num_heads,
#         dropout=0.0,
#         mha_type="relmha",
#         kernel_regularizer=L2,
#         bias_regularizer=L2,
#         name="mhsa_module",
#         **kwargs,
#     ):
#         super(MHSAModule, self).__init__(name=name, **kwargs)
#         self.ln = tf.keras.layers.LayerNormalization(
#             name=f"{name}_ln",
#             gamma_regularizer=kernel_regularizer,
#             beta_regularizer=bias_regularizer,
#         )
#         if mha_type == "relmha":
#             self.mha = RelPositionMultiHeadAttention(
#                 name=f"{name}_mhsa",
#                 head_size=head_size,
#                 num_heads=num_heads,
#                 kernel_regularizer=kernel_regularizer,
#                 bias_regularizer=bias_regularizer,
#             )
#         elif mha_type == "mha":
#             self.mha = MultiHeadAttention(
#                 name=f"{name}_mhsa",
#                 head_size=head_size,
#                 num_heads=num_heads,
#                 kernel_regularizer=kernel_regularizer,
#                 bias_regularizer=bias_regularizer,
#             )
#         else:
#             raise ValueError("mha_type must be either 'mha' or 'relmha'")
#         self.do = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout")
#         self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.5)
#         self.mha_type = mha_type
#         self.supports_masking = True # RAJOUT

#     def call(
#         self,
#         inputs,
#         training=None,
#         mask=None,
#         **kwargs,
#     ):
#         inputs, pos = inputs  # pos is positional encoding
#         outputs = self.ln(inputs, training=training)
#         if self.mha_type == "relmha":
#             outputs = self.mha([outputs, outputs, outputs, pos], training=training, mask=mask)
#         else:
#             outputs = outputs + pos
#             outputs = self.mha([outputs, outputs, outputs], training=training, mask=mask)
#         outputs = self.do(outputs, training=training)
#         outputs = self.res_add([inputs, outputs])
#         return outputs

#     def get_config(self):
#         conf = super(MHSAModule, self).get_config()
#         conf.update({"mha_type": self.mha_type})
#         conf.update(self.ln.get_config())
#         conf.update(self.mha.get_config())
#         conf.update(self.do.get_config())
#         conf.update(self.res_add.get_config())
#         return conf


# class ConvModule(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         input_dim,
#         kernel_size=32,
#         dropout=0.0,
#         depth_multiplier=1,
#         kernel_regularizer=L2,
#         bias_regularizer=L2,
#         name="conv_module",
#         **kwargs,
#     ):
#         super(ConvModule, self).__init__(name=name, **kwargs)
#         self.ln = tf.keras.layers.LayerNormalization()
#         self.pw_conv_1 = tf.keras.layers.Conv1D(
#             filters=2 * input_dim,
#             kernel_size=1,
#             strides=1,
#             padding="valid",
#             name=f"{name}_pw_conv_1",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.glu = GLU(name=f"{name}_glu")
#         self.dw_conv = CausalDWConv1D(
#             kernel_size=kernel_size,
#             name=f"{name}_dw_conv",
#             depth_multiplier = depth_multiplier,
#         )
#         self.bn = tf.keras.layers.BatchNormalization(
#             name=f"{name}_bn",
#             gamma_regularizer=kernel_regularizer,
#             beta_regularizer=bias_regularizer,
#         )
#         self.swish = tf.keras.layers.Activation(
#             tf.nn.swish,
#             name=f"{name}_swish_activation",
#         )
#         self.pw_conv_2 = tf.keras.layers.Conv1D(
#             filters=input_dim,
#             kernel_size=1,
#             strides=1,
#             padding="valid",
#             name=f"{name}_pw_conv_2",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.do = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout")
#         self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.5)

#     def call(
#         self,
#         inputs,
#         training=None,
#         **kwargs,
#     ):
#         outputs = self.ln(inputs, training=training)
#         B, T, E = shape_list(outputs)
#         outputs = tf.reshape(outputs, [B, T, E]) # [B, T, 1, E]
#         outputs = self.pw_conv_1(outputs, training=training)
#         outputs = self.glu(outputs)
#         outputs = self.dw_conv(outputs, training=training)
#         outputs = self.bn(outputs, training=training)
#         outputs = self.swish(outputs)
#         outputs = self.pw_conv_2(outputs, training=training)
#         outputs = tf.reshape(outputs, [B, T, E]) #
#         outputs = self.do(outputs, training=training)
#         outputs = self.res_add([inputs, outputs])
#         return outputs

#     def get_config(self):
#         conf = super(ConvModule, self).get_config()
#         conf.update(self.ln.get_config())
#         conf.update(self.pw_conv_1.get_config())
#         conf.update(self.glu.get_config())
#         conf.update(self.dw_conv.get_config())
#         conf.update(self.bn.get_config())
#         conf.update(self.swish.get_config())
#         conf.update(self.pw_conv_2.get_config())
#         conf.update(self.do.get_config())
#         conf.update(self.res_add.get_config())
#         return conf
    
    

# class CausalDWConv1D(tf.keras.layers.Layer):
#     def __init__(self,
#         kernel_size=17,
#         dilation_rate=1,
#         use_bias=False,
#         depthwise_initializer=tf.keras.initializers.glorot_uniform(seed=SEED),
#         depth_multiplier = 1,
#         name='', **kwargs):
#         super().__init__(name=name,**kwargs)
#         self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
#         self.dw_conv = tf.keras.layers.DepthwiseConv1D(
#                             kernel_size,
#                             strides=1,
#                             dilation_rate=dilation_rate,
#                             depth_multiplier=depth_multiplier,
#                             padding='valid',
#                             use_bias=use_bias,
#                             depthwise_initializer=depthwise_initializer,
#                             name=name + '_dwconv')
#         self.supports_masking = True

#     def call(self, inputs):
#         x = self.causal_pad(inputs)
#         x = self.dw_conv(x)
#         return x
    
# class ConformerBlock(tf.keras.layers.Layer):
#     def __init__(
#         self,
#         input_dim,
#         dropout=0.0,
#         fc_factor=0.5,
#         head_size=36,
#         num_heads=4,
#         mha_type="relmha",
#         kernel_size=32,
#         depth_multiplier=1,
#         kernel_regularizer=L2,
#         bias_regularizer=L2,
#         name="conformer_block",
#         **kwargs,
#     ):
#         super(ConformerBlock, self).__init__(name=name, **kwargs)
#         self.ffm1 = FFModule(
#             input_dim=input_dim,
#             dropout=dropout,
#             fc_factor=fc_factor,
#             name=f"{name}_ff_module_1",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.mhsam = MHSAModule(
#             mha_type=mha_type,
#             head_size=head_size,
#             num_heads=num_heads,
#             dropout=dropout,
#             name=f"{name}_mhsa_module",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.convm = ConvModule(
#             input_dim=input_dim,
#             kernel_size=kernel_size,
#             dropout=dropout,
#             name=f"{name}_conv_module",
#             depth_multiplier=depth_multiplier,
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.ffm2 = FFModule(
#             input_dim=input_dim,
#             dropout=dropout,
#             fc_factor=fc_factor,
#             name=f"{name}_ff_module_2",
#             kernel_regularizer=kernel_regularizer,
#             bias_regularizer=bias_regularizer,
#         )
#         self.ln = tf.keras.layers.LayerNormalization(
#             name=f"{name}_ln",
#             gamma_regularizer=kernel_regularizer,
#             beta_regularizer=kernel_regularizer,
#         )
#         self.supports_masking = True # RAJOUT

#     def call(
#         self,
#         inputs,
#         training=None,
#         mask=None,
#         **kwargs,
#     ):
#         inputs, pos = inputs  # pos is positional encoding
#         outputs = self.ffm1(inputs, training=training, **kwargs)
#         outputs = self.mhsam([outputs, pos], training=training, mask=mask, **kwargs)
#         outputs = self.convm(outputs, training=training, **kwargs)
#         outputs = self.ffm2(outputs, training=training, **kwargs)
#         outputs = self.ln(outputs, training=training)
#         return outputs

#     def get_config(self):
#         conf = super(ConformerBlock, self).get_config()
#         conf.update(self.ffm1.get_config())
#         conf.update(self.mhsam.get_config())
#         conf.update(self.convm.get_config())
#         conf.update(self.ffm2.get_config())
#         conf.update(self.ln.get_config())
#         return conf

# def get_attention_mask(x_inp):
#     # Attention Mask
#     attention_mask = tf.math.count_nonzero(x_inp, axis=[2], keepdims=True, dtype=tf.int32)
#     attention_mask = tf.math.count_nonzero(attention_mask, axis=[2], keepdims=True)
#     attention_mask = tf.broadcast_to(attention_mask, [attention_mask.shape[0], attention_mask.shape[1], attention_mask.shape[1]])
#     return attention_mask

In [14]:
import typing
seed_everything()
def shape_list(x, out_type=tf.int32):
    """Deal with dynamic shape in tensorflow cleanly."""
    static = x.shape.as_list()
    dynamic = tf.shape(x, out_type=out_type)
    return [dynamic[i] if s is None else s for i, s in enumerate(static)]


def get_shape_invariants(tensor):
    shapes = shape_list(tensor)
    return tf.TensorShape([i if isinstance(i, int) else None for i in shapes])


def get_float_spec(tensor):
    shape = get_shape_invariants(tensor)
    return tf.TensorSpec(shape, dtype=tf.float32)

class GLU(tf.keras.layers.Layer):
    def __init__(
        self,
        axis=-1,
        name="glu_activation",
        **kwargs,
    ):
        super(GLU, self).__init__(name=name, **kwargs)
        self.axis = axis

    def call(
        self,
        inputs,
        **kwargs,
    ):
        a, b = tf.split(inputs, 2, axis=self.axis)
        b = tf.nn.sigmoid(b)
        return tf.multiply(a, b)

    def get_config(self):
        conf = super(GLU, self).get_config()
        conf.update({"axis": self.axis})
        return conf


class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(
        self,
        num_heads,
        head_size,
        output_size: int = None,
        dropout: float = 0.0,
        use_projection_bias: bool = True,
        return_attn_coef: bool = False,
        kernel_initializer: typing.Union[str, typing.Callable] = tf.keras.initializers.glorot_uniform(seed=SEED),
        kernel_regularizer: typing.Union[str, typing.Callable] = None,
        kernel_constraint: typing.Union[str, typing.Callable] = None,
        bias_initializer: typing.Union[str, typing.Callable] = "zeros",
        bias_regularizer: typing.Union[str, typing.Callable] = None,
        bias_constraint: typing.Union[str, typing.Callable] = None,
        **kwargs,
    ):
        super(MultiHeadAttention, self).__init__(**kwargs)

        if output_size is not None and output_size < 1:
            raise ValueError("output_size must be a positive number")

        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.kernel_regularizer = tf.keras.regularizers.get(kernel_regularizer)
        self.kernel_constraint = tf.keras.constraints.get(kernel_constraint)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        self.bias_regularizer = tf.keras.regularizers.get(bias_regularizer)
        self.bias_constraint = tf.keras.constraints.get(bias_constraint)

        self.head_size = head_size
        self.num_heads = num_heads
        self.output_size = output_size
        self.use_projection_bias = use_projection_bias
        self.return_attn_coef = return_attn_coef

        self.dropout = tf.keras.layers.Dropout(dropout, name="dropout")
        self._droput_rate = dropout
        self.supports_masking = True # RAJOUT

    def build(
        self,
        input_shape,
    ):
        num_query_features = input_shape[0][-1]
        num_key_features = input_shape[1][-1]
        num_value_features = input_shape[2][-1] if len(input_shape) > 2 else num_key_features
        output_size = self.output_size if self.output_size is not None else num_value_features
        self.query_kernel = self.add_weight(
            name="query_kernel",
            shape=[self.num_heads, num_query_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.key_kernel = self.add_weight(
            name="key_kernel",
            shape=[self.num_heads, num_key_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.value_kernel = self.add_weight(
            name="value_kernel",
            shape=[self.num_heads, num_value_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.projection_kernel = self.add_weight(
            name="projection_kernel",
            shape=[self.num_heads, self.head_size, output_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        if self.use_projection_bias:
            self.projection_bias = self.add_weight(
                name="projection_bias",
                shape=[output_size],
                initializer=self.bias_initializer,
                regularizer=self.bias_regularizer,
                constraint=self.bias_constraint,
            )
        else:
            self.projection_bias = None

    def call_qkv(
        self,
        query,
        key,
        value,
        training=None,
    ):
        # verify shapes
        if key.shape[-2] != value.shape[-2]:
            raise ValueError(
                "the number of elements in 'key' must be equal to " "the same as the number of elements in 'value'"
            )
        # Linear transformations
        query = tf.einsum("...NI,HIO->...NHO", query, self.query_kernel)
        key = tf.einsum("...MI,HIO->...MHO", key, self.key_kernel)
        value = tf.einsum("...MI,HIO->...MHO", value, self.value_kernel)

        return query, key, value

    def call_attention(
        self,
        query,
        key,
        value,
        logits,
        training=None,
        mask=None,
        attention_mask=None,
    ):
        # mask = attention mask with shape [B, Tquery, Tkey] with 1 is for positions we want to attend, 0 for masked
        if attention_mask is not None:
            if len(attention_mask.shape) < 2: #was written mask
                raise ValueError("'mask' must have at least 2 dimensions")
            if query.shape[-3] != attention_mask.shape[-2]:
                raise ValueError("mask's second to last dimension must be equal to " "the number of elements in 'query'")
            if key.shape[-3] != attention_mask.shape[-1]:
                raise ValueError("mask's last dimension must be equal to the number of elements in 'key'")
        # apply mask
        if attention_mask is not None:
            attention_mask = tf.cast(attention_mask, tf.float32)

            # possibly expand on the head dimension so broadcasting works
            if len(attention_mask.shape) != len(logits.shape):
                attention_mask = tf.expand_dims(attention_mask, -3)

            logits += -10e9 * (1.0 - attention_mask)

        attn_coef = tf.nn.softmax(logits)

        # attention dropout
        attn_coef_dropout = self.dropout(attn_coef, training=training)

        # attention * value
        multihead_output = tf.einsum("...HNM,...MHI->...NHI", attn_coef_dropout, value)

        # Run the outputs through another linear projection layer. Recombining heads
        # is automatically done.
        output = tf.einsum("...NHI,HIO->...NO", multihead_output, self.projection_kernel)

        if self.projection_bias is not None:
            output += self.projection_bias

        return output, attn_coef

    def call(
        self,
        inputs,
        training=None,
        mask=None,
        attention_mask=None,
        **kwargs,
    ):
        query, key, value = inputs

        query, key, value = self.call_qkv(query, key, value, training=training)

        # Scale dot-product, doing the division to either query or key
        # instead of their product saves some computation
        depth = tf.constant(self.head_size, dtype=tf.float32)
        query /= tf.sqrt(depth)

        # Calculate dot product attention
        logits = tf.einsum("...NHO,...MHO->...HNM", query, key)

        output, attn_coef = self.call_attention(query, key, value, logits, training=training, mask=mask, attention_mask=attention_mask)

        if self.return_attn_coef:
            return output, attn_coef
        else:
            return output

    def compute_output_shape(
        self,
        input_shape,
    ):
        num_value_features = input_shape[2][-1] if len(input_shape) > 2 else input_shape[1][-1]
        output_size = self.output_size if self.output_size is not None else num_value_features

        output_shape = input_shape[0][:-1] + (output_size,)

        if self.return_attn_coef:
            num_query_elements = input_shape[0][-2]
            num_key_elements = input_shape[1][-2]
            attn_coef_shape = input_shape[0][:-2] + (
                self.num_heads,
                num_query_elements,
                num_key_elements,
            )

            return output_shape, attn_coef_shape
        else:
            return output_shape

    def get_config(self):
        config = super().get_config()

        config.update(
            head_size=self.head_size,
            num_heads=self.num_heads,
            output_size=self.output_size,
            dropout=self._droput_rate,
            use_projection_bias=self.use_projection_bias,
            return_attn_coef=self.return_attn_coef,
            kernel_initializer=tf.keras.initializers.serialize(self.kernel_initializer),
            kernel_regularizer=tf.keras.regularizers.serialize(self.kernel_regularizer),
            kernel_constraint=tf.keras.constraints.serialize(self.kernel_constraint),
            bias_initializer=tf.keras.initializers.serialize(self.bias_initializer),
            bias_regularizer=tf.keras.regularizers.serialize(self.bias_regularizer),
            bias_constraint=tf.keras.constraints.serialize(self.bias_constraint),
        )

        return config


class RelPositionMultiHeadAttention(MultiHeadAttention):
    def build(
        self,
        input_shape,
    ):
        num_pos_features = input_shape[-1][-1]
        self.pos_kernel = self.add_weight(
            name="pos_kernel",
            shape=[self.num_heads, num_pos_features, self.head_size],
            initializer=self.kernel_initializer,
            regularizer=self.kernel_regularizer,
            constraint=self.kernel_constraint,
        )
        self.pos_bias_u = self.add_weight(
            name="pos_bias_u",
            shape=[self.num_heads, self.head_size],
            regularizer=self.kernel_regularizer,
            initializer=self.kernel_initializer,
            constraint=self.kernel_constraint,
        )
        self.pos_bias_v = self.add_weight(
            name="pos_bias_v",
            shape=[self.num_heads, self.head_size],
            regularizer=self.kernel_regularizer,
            initializer=self.kernel_initializer,
            constraint=self.kernel_constraint,
        )
        super(RelPositionMultiHeadAttention, self).build(input_shape[:-1])

    @staticmethod
    def relative_shift(x):
        x_shape = tf.shape(x)
        x = tf.pad(x, [[0, 0], [0, 0], [0, 0], [1, 0]])
        x = tf.reshape(x, [x_shape[0], x_shape[1], x_shape[3] + 1, x_shape[2]])
        x = tf.reshape(x[:, :, 1:, :], x_shape)
        return x

    def call(
        self,
        inputs,
        training=None,
        mask=None,
        attention_mask=None,
        **kwargs,
    ):
        query, key, value, pos = inputs

        query, key, value = self.call_qkv(query, key, value, training=training)

        pos = tf.einsum("...MI,HIO->...MHO", pos, self.pos_kernel)

        query_with_u = query + self.pos_bias_u
        query_with_v = query + self.pos_bias_v

        logits_with_u = tf.einsum("...NHO,...MHO->...HNM", query_with_u, key)
        logits_with_v = tf.einsum("...NHO,...MHO->...HNM", query_with_v, pos)
        logits_with_v = self.relative_shift(logits_with_v)

        logits = logits_with_u + logits_with_v[:, :, :, : tf.shape(logits_with_u)[3]]

        depth = tf.constant(self.head_size, dtype=tf.float32)
        logits /= tf.sqrt(depth)

        output, attn_coef = self.call_attention(query, key, value, logits, training=training, mask=mask, attention_mask=attention_mask)

        if self.return_attn_coef:
            return output, attn_coef
        else:
            return output


class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(
        self,
        alpha: int = 1,
        beta: int = 0,
        name="positional_encoding",
        **kwargs,
    ):
        super().__init__(trainable=False, name=name, **kwargs)
        self.alpha = alpha
        self.beta = beta
        self.supports_masking = True # RAJOUT

    def build(
        self,
        input_shape,
    ):
        dmodel = input_shape[-1]
        assert dmodel % 2 == 0, f"Input last dim must be even: {dmodel}"

    @staticmethod
    def encode(
        max_len,
        dmodel,
    ):
        pos = tf.expand_dims(tf.range(max_len - 1, -1, -1.0, dtype=tf.float32), axis=1)
        index = tf.expand_dims(tf.range(0, dmodel, dtype=tf.float32), axis=0)

        pe = pos * (1 / tf.pow(10000.0, (2 * (index // 2)) / dmodel))

        # Sin cos will be [max_len, size // 2]
        # we add 0 between numbers by using padding and reshape
        sin = tf.pad(tf.expand_dims(tf.sin(pe[:, 0::2]), -1), [[0, 0], [0, 0], [0, 1]], mode="CONSTANT", constant_values=0)
        sin = tf.reshape(sin, [max_len, dmodel])
        cos = tf.pad(tf.expand_dims(tf.cos(pe[:, 1::2]), -1), [[0, 0], [0, 0], [1, 0]], mode="CONSTANT", constant_values=0)
        cos = tf.reshape(cos, [max_len, dmodel])
        # Then add sin and cos, which results in [time, size]
        pe = tf.add(sin, cos)
        return tf.expand_dims(pe, axis=0)  # [1, time, size]

    def call(
        self,
        inputs,
        **kwargs,
    ):
        # inputs shape [B, T, V]
        _, max_len, dmodel = shape_list(inputs)
        pe = self.encode(max_len * self.alpha + self.beta, dmodel)
        return tf.cast(pe, dtype=inputs.dtype)

    def get_config(self):
        conf = super().get_config()
        conf.update({"alpha": self.alpha, "beta": self.beta})
        return conf


class PositionalEncodingConcat(PositionalEncoding):
    def build(
        self,
        input_shape,
    ):
        dmodel = input_shape[-1]
        assert dmodel % 2 == 0, f"Input last dim must be even: {dmodel}"

    @staticmethod
    def encode(
        max_len,
        dmodel,
    ):
        pos = tf.range(max_len - 1, -1, -1.0, dtype=tf.float32)

        index = tf.range(0, dmodel, 2.0, dtype=tf.float32)
        index = 1 / tf.pow(10000.0, (index / dmodel))

        sinusoid = tf.einsum("i,j->ij", pos, index)
        pos = tf.concat([tf.sin(sinusoid), tf.cos(sinusoid)], axis=-1)

        return tf.expand_dims(pos, axis=0)

    def call(
        self,
        inputs,
        **kwargs,
    ):
        # inputs shape [B, T, V]
        _, max_len, dmodel = shape_list(inputs)
        pe = self.encode(max_len * self.alpha + self.beta, dmodel)
        return tf.cast(pe, dtype=inputs.dtype)



L2 = tf.keras.regularizers.l2(1e-6)


class FFModule(tf.keras.layers.Layer):
    def __init__(
        self,
        input_dim,
        dropout=0.0,
        fc_factor=0.5,
        kernel_regularizer=L2,
        bias_regularizer=L2,
        name="ff_module",
        **kwargs,
    ):
        super(FFModule, self).__init__(name=name, **kwargs)
        self.fc_factor = fc_factor
        self.ln = tf.keras.layers.LayerNormalization(
            name=f"{name}_ln",
            gamma_regularizer=kernel_regularizer,
            beta_regularizer=bias_regularizer,
        )
        self.ffn1 = tf.keras.layers.Dense(
            4 * input_dim,
            name=f"{name}_dense_1",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.swish = tf.keras.layers.Activation(tf.nn.swish, name=f"{name}_swish_activation")
        self.do1 = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout_1")
        self.ffn2 = tf.keras.layers.Dense(
            input_dim,
            name=f"{name}_dense_2",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.do2 = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout_2")
        self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.4)
        self.supports_masking = True # RAJOUT

    def call(
        self,
        inputs,
        training=None,
        **kwargs,
    ):
        outputs = self.ln(inputs, training=training)
        outputs = self.ffn1(outputs, training=training)
        outputs = self.swish(outputs)
        outputs = self.do1(outputs, training=training)
        outputs = self.ffn2(outputs, training=training)
        outputs = self.do2(outputs, training=training)
        outputs = self.res_add([inputs, self.fc_factor * outputs])
        return outputs

    def get_config(self):
        conf = super(FFModule, self).get_config()
        conf.update({"fc_factor": self.fc_factor})
        conf.update(self.ln.get_config())
        conf.update(self.ffn1.get_config())
        conf.update(self.swish.get_config())
        conf.update(self.do1.get_config())
        conf.update(self.ffn2.get_config())
        conf.update(self.do2.get_config())
        conf.update(self.res_add.get_config())
        return conf


class MHSAModule(tf.keras.layers.Layer):
    def __init__(
        self,
        head_size,
        num_heads,
        dropout=0.0,
        mha_type="relmha",
        kernel_regularizer=L2,
        bias_regularizer=L2,
        name="mhsa_module",
        **kwargs,
    ):
        super(MHSAModule, self).__init__(name=name, **kwargs)
        self.ln = tf.keras.layers.LayerNormalization(
            name=f"{name}_ln",
            gamma_regularizer=kernel_regularizer,
            beta_regularizer=bias_regularizer,
        )
        if mha_type == "relmha":
            self.mha = RelPositionMultiHeadAttention(
                name=f"{name}_mhsa",
                head_size=head_size,
                num_heads=num_heads,
                kernel_regularizer=kernel_regularizer,
                bias_regularizer=bias_regularizer,
            )
        elif mha_type == "mha":
            self.mha = MultiHeadAttention(
                name=f"{name}_mhsa",
                head_size=head_size,
                num_heads=num_heads,
                kernel_regularizer=kernel_regularizer,
                bias_regularizer=bias_regularizer,
            )
        else:
            raise ValueError("mha_type must be either 'mha' or 'relmha'")
        self.do = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout")
        self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.5)
        self.mha_type = mha_type
        self.supports_masking = True # RAJOUT

    def call(
        self,
        inputs,
        training=None,
        mask=None,
        attention_mask=None,
        **kwargs,
    ):
        inputs, pos = inputs  # pos is positional encoding
        outputs = self.ln(inputs, training=training)
        if self.mha_type == "relmha":
            outputs = self.mha([outputs, outputs, outputs, pos], training=training, mask=mask, attention_mask=attention_mask)
        else:
            outputs = outputs + pos
            outputs = self.mha([outputs, outputs, outputs], training=training, mask=mask, attention_mask=attention_mask)
        outputs = self.do(outputs, training=training)
        outputs = self.res_add([inputs, outputs])
        return outputs

    def get_config(self):
        conf = super(MHSAModule, self).get_config()
        conf.update({"mha_type": self.mha_type})
        conf.update(self.ln.get_config())
        conf.update(self.mha.get_config())
        conf.update(self.do.get_config())
        conf.update(self.res_add.get_config())
        return conf


class ConvModule(tf.keras.layers.Layer):
    def __init__(
        self,
        input_dim,
        kernel_size=32,
        dropout=0.0,
        depth_multiplier=1,
        kernel_regularizer=L2,
        bias_regularizer=L2,
        name="conv_module",
        **kwargs,
    ):
        super(ConvModule, self).__init__(name=name, **kwargs)
        self.ln = tf.keras.layers.LayerNormalization()
        self.pw_conv_1 = tf.keras.layers.Conv1D(
            filters=2 * input_dim,
            kernel_size=1,
            strides=1,
            padding="valid",
            name=f"{name}_pw_conv_1",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.glu = GLU(name=f"{name}_glu")
        self.dw_conv = CausalDWConv1D(
            kernel_size=kernel_size,
            name=f"{name}_dw_conv",
            depth_multiplier = depth_multiplier,
        )
        self.bn = tf.keras.layers.BatchNormalization(
            name=f"{name}_bn",
            gamma_regularizer=kernel_regularizer,
            beta_regularizer=bias_regularizer,
        )
        self.swish = tf.keras.layers.Activation(
            tf.nn.swish,
            name=f"{name}_swish_activation",
        )
        self.pw_conv_2 = tf.keras.layers.Conv1D(
            filters=input_dim,
            kernel_size=1,
            strides=1,
            padding="valid",
            name=f"{name}_pw_conv_2",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.do = tf.keras.layers.Dropout(dropout, name=f"{name}_dropout")
        self.res_add = tfa.layers.StochasticDepth(name=f"{name}_add", survival_probability=0.5)

    def call(
        self,
        inputs,
        training=None,
        **kwargs,
    ):
        outputs = self.ln(inputs, training=training)
        B, T, E = shape_list(outputs)
        outputs = tf.reshape(outputs, [B, T, E]) # [B, T, 1, E]
        outputs = self.pw_conv_1(outputs, training=training)
        outputs = self.glu(outputs)
        outputs = self.dw_conv(outputs, training=training)
        outputs = self.bn(outputs, training=training)
        outputs = self.swish(outputs)
        outputs = self.pw_conv_2(outputs, training=training)
        outputs = tf.reshape(outputs, [B, T, E]) #
        outputs = self.do(outputs, training=training)
        outputs = self.res_add([inputs, outputs])
        return outputs

    def get_config(self):
        conf = super(ConvModule, self).get_config()
        conf.update(self.ln.get_config())
        conf.update(self.pw_conv_1.get_config())
        conf.update(self.glu.get_config())
        conf.update(self.dw_conv.get_config())
        conf.update(self.bn.get_config())
        conf.update(self.swish.get_config())
        conf.update(self.pw_conv_2.get_config())
        conf.update(self.do.get_config())
        conf.update(self.res_add.get_config())
        return conf



class CausalDWConv1D(tf.keras.layers.Layer):
    def __init__(self,
        kernel_size=17,
        dilation_rate=1,
        use_bias=False,
        depthwise_initializer=tf.keras.initializers.glorot_uniform(seed=SEED),
        depth_multiplier = 1,
        name='', **kwargs):
        super().__init__(name=name,**kwargs)
        self.causal_pad = tf.keras.layers.ZeroPadding1D((dilation_rate*(kernel_size-1),0),name=name + '_pad')
        self.dw_conv = tf.keras.layers.DepthwiseConv1D(
                            kernel_size,
                            strides=1,
                            dilation_rate=dilation_rate,
                            depth_multiplier=depth_multiplier,
                            padding='valid',
                            use_bias=use_bias,
                            depthwise_initializer=depthwise_initializer,
                            name=name + '_dwconv')
        self.supports_masking = True

    def call(self, inputs):
        x = self.causal_pad(inputs)
        x = self.dw_conv(x)
        return x

class ConformerBlock(tf.keras.layers.Layer):
    def __init__(
        self,
        input_dim,
        dropout=0.0,
        fc_factor=0.5,
        head_size=36,
        num_heads=4,
        mha_type="relmha",
        kernel_size=32,
        depth_multiplier=1,
        kernel_regularizer=L2,
        bias_regularizer=L2,
        name="conformer_block",
        **kwargs,
    ):
        super(ConformerBlock, self).__init__(name=name, **kwargs)
        self.ffm1 = FFModule(
            input_dim=input_dim,
            dropout=dropout,
            fc_factor=fc_factor,
            name=f"{name}_ff_module_1",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.mhsam = MHSAModule(
            mha_type=mha_type,
            head_size=head_size,
            num_heads=num_heads,
            dropout=dropout,
            name=f"{name}_mhsa_module",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.convm = ConvModule(
            input_dim=input_dim,
            kernel_size=kernel_size,
            dropout=dropout,
            name=f"{name}_conv_module",
            depth_multiplier=depth_multiplier,
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.ffm2 = FFModule(
            input_dim=input_dim,
            dropout=dropout,
            fc_factor=fc_factor,
            name=f"{name}_ff_module_2",
            kernel_regularizer=kernel_regularizer,
            bias_regularizer=bias_regularizer,
        )
        self.ln = tf.keras.layers.LayerNormalization(
            name=f"{name}_ln",
            gamma_regularizer=kernel_regularizer,
            beta_regularizer=kernel_regularizer,
        )
        self.supports_masking = True # RAJOUT

    def call(
        self,
        inputs,
        training=None,
        mask=None,
        attention_mask=None,
        **kwargs,
    ):
        inputs, pos = inputs  # pos is positional encoding
        outputs = self.ffm1(inputs, training=training, **kwargs)
        outputs = self.mhsam([outputs, pos], training=training, mask=mask, attention_mask=attention_mask, **kwargs)
        outputs = self.convm(outputs, training=training, **kwargs)
        outputs = self.ffm2(outputs, training=training, **kwargs)
        outputs = self.ln(outputs, training=training)
        return outputs

    def get_config(self):
        conf = super(ConformerBlock, self).get_config()
        conf.update(self.ffm1.get_config())
        conf.update(self.mhsam.get_config())
        conf.update(self.convm.get_config())
        conf.update(self.ffm2.get_config())
        conf.update(self.ln.get_config())
        return conf


def get_attention_mask(x_inp, mask_value):
    padding_mask = tf.reduce_sum(x_inp, axis=-1)
    padding_mask = tf.cast(tf.math.equal(padding_mask, mask_value), tf.float32)
    padding_mask = 1 - padding_mask
    padding_mask = padding_mask[:, tf.newaxis, :]
    return padding_mask

In [15]:
def CTCLoss(labels, logits):
    label_length = tf.reduce_sum(tf.cast(labels != pad_token_idx, tf.int32), axis=-1)
    logit_length = tf.ones(tf.shape(logits)[0], dtype=tf.int32) * tf.shape(logits)[1]
    
    loss = classic_ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
        )
    '''
    loss = tf.nn.ctc_loss(
            labels=labels,
            logits=logits,
            label_length=label_length,
            logit_length=logit_length,
            blank_index=pad_token_idx,
            logits_time_major=False
        )
    '''
    loss = tf.reduce_mean(loss)
    return loss

In [16]:
# Config param

INPUT_SHAPE = [128, 276] #format d'entrée
dim = 256 # Embedding dimension
num_blocs = 12
dropout_cformer = 0.0 # Dropout applied in each module of conformer block, in different location in modules
num_heads = 8
head_size = dim // num_heads # head_size * num_heads should be equal to dim
depth_multiplier = 1 # didn't try to modify it but I think it will compile anyway
kernel_size = 3 # kernel size of Conv module, specially the dephwiseConv1d

In [17]:
from tensorflow.keras.layers import SpatialDropout1D

def get_model(dim = dim, num_blocs = num_blocs):
    inp = tf.keras.Input(INPUT_SHAPE, name="input")

#         attention_mask = get_attention_mask(inp, value_pad=-100.0)
#         mask = tf.keras.layers.Masking(mask_value=-100.0, input_shape=INPUT_SHAPE)

#         x = mask(inp)
    x = tf.keras.layers.Dense(dim, use_bias=False,name='stem_conv')(inp)
    pe = PositionalEncoding()
    pex = pe(x)
        
   # x = SpatialDropout1D(0.2, name='spatial_dropout_pe')(x)  # DO spatial

    # x = tf.keras.layers.Dropout(0.4)(x) # you can activate that sku sku dropout if you want to

    conf_blocks = []
    for i in range(num_blocs):
        name = f'Conf_block_{i}'
        conf_block= ConformerBlock(input_dim=dim,
                                   head_size=head_size,
                                   dropout=dropout_cformer,
                                   num_heads=num_heads,
                                   depth_multiplier=depth_multiplier,
                                   kernel_size=kernel_size,
                                   name=name)
        conf_blocks.append(conf_block)

    for cblock in conf_blocks:
        x = cblock([x, pex])

   # x = SpatialDropout1D(0.2, name='spatial_dropout_pe')(x)  # DO spatial
    x = tf.keras.layers.Dense(dim*2,activation="relu",name='top_conv')(x)
    x = tf.keras.layers.Dropout(0.7)(x)
    x = tf.keras.layers.Dense(len(char_to_num))(x)

    model = tf.keras.Model(inp, x)

    loss = CTCLoss

    #Adam Optimizer
    optimizer = tfa.optimizers.RectifiedAdam(sma_threshold=4)
    optimizer = tfa.optimizers.Lookahead(optimizer, sync_period=5)

    model.compile(loss=loss, optimizer=optimizer)

    return model

In [18]:
def num_to_char_fn(y):
    return [num_to_char.get(x, "") for x in y]

@tf.function()
def decode_phrase(pred):
    x = tf.argmax(pred, axis=1)
    diff = tf.not_equal(x[:-1], x[1:])
    adjacent_indices = tf.where(diff)[:, 0]
    x = tf.gather(x, adjacent_indices)
    mask = x != pad_token_idx
    x = tf.boolean_mask(x, mask, axis=0)
    return x

# A utility function to decode the output of the network
def decode_batch_predictions(pred):
    output_text = []
    for result in pred:
        result = "".join(num_to_char_fn(decode_phrase(result).numpy()))
        output_text.append(result)
    return output_text

# Inference

In [19]:
# Clear all models in GPU
tf.keras.backend.clear_session()
seed_everything(seed=SEED)
# Get new fresh model
model = get_model()
# model(batch[0])
# Sanity Check
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 128, 276)]   0           []                               
                                                                                                  
 stem_conv (Dense)              (None, 128, 256)     70656       ['input[0][0]']                  
                                                                                                  
 positional_encoding (Positiona  (1, 128, 256)       0           ['stem_conv[0][0]']              
 lEncoding)                                                                                       
                                                                                                  
 Conf_block_0 (ConformerBlock)  (None, 128, 256)     1581312     ['stem_conv[0][0]',          

In [20]:
model.load_weights('/kaggle/input/zzzzzz/model_epoch_132_val_loss_17.1502.h5')
print(f'Successfully Loaded Pretrained Weights')

Successfully Loaded Pretrained Weights


In [21]:
#model.get_weight_paths()
# Model Layer Names
for l in model.layers:
    print(l.name)

input
stem_conv
positional_encoding
Conf_block_0
Conf_block_1
Conf_block_2
Conf_block_3
Conf_block_4
Conf_block_5
Conf_block_6
Conf_block_7
Conf_block_8
Conf_block_9
Conf_block_10
Conf_block_11
top_conv
dropout
dense


In [22]:
class TFLiteModel(tf.Module):
    def __init__(self, model):
        super(TFLiteModel, self).__init__()
        self.model = model
    
    @tf.function(input_signature=[tf.TensorSpec(shape=[None, len(SEL_COLS)], dtype=tf.float32, name='inputs')])
    def __call__(self, inputs, training=False):
        # Preprocess Data
        x = tf.cast(inputs, tf.float32)
        x = x[None]
        x = tf.cond(tf.shape(x)[1] == 0, lambda: tf.zeros((1, 1, len(SEL_COLS))), lambda: tf.identity(x))
        x = x[0]
        x = pre_process0(x)
        x = pre_process1(*x)
        x = tf.reshape(x, INPUT_SHAPE)
        x = x[None]
        x = self.model(x, training=False)
        x = x[0]
        x = decode_phrase(x)
        x = tf.cond(tf.shape(x)[0] == 0, lambda: tf.zeros(1, tf.int64), lambda: tf.identity(x))
        x = tf.one_hot(x, 59)
        return {'outputs': x}

tflitemodel_base = TFLiteModel(model)
tflitemodel_base(frames)["outputs"].shape

TensorShape([13, 59])

In [23]:
keras_model_converter = tf.lite.TFLiteConverter.from_keras_model(tflitemodel_base)
keras_model_converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS]#, tf.lite.OpsSet.SELECT_TF_OPS]
keras_model_converter.optimizations = [tf.lite.Optimize.DEFAULT]
keras_model_converter.target_spec.supported_types = [tf.float16]
tflite_model = keras_model_converter.convert()
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)
    

In [24]:
# Add selected_columns json to only select specific columns from input frames
with open('inference_args.json', "w") as f:
    json.dump({"selected_columns" : SEL_COLS}, f)

In [25]:
# Zip Model
!zip submission.zip  './model.tflite' './inference_args.json'

  adding: model.tflite (deflated 10%)
  adding: inference_args.json (deflated 83%)
